In [1]:
import pandas as pd
import numpy as np

In [2]:
df_movies = pd.read_csv('movies.csv')
df_ratings = pd.read_csv('ratings.csv')

In [3]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
genres_set = set()
df_movies_profile = df_movies.copy()
for row in range(len(df_movies)):
    genres_column_list = df_movies.loc[row,"genres"].split("|")
    for genre in genres_column_list:
        if genre not in df_movies_profile.columns:
            df_movies_profile[genre] = 0
        df_movies_profile.loc[row,genre] = 1
df_movies_profile["norm"] = np.sqrt(np.sum(df_movies_profile.iloc[:,3:],axis=1))

In [6]:
df_movies_profile.head()

,movieId,title,genres,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),norm
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,2.236068
1,2,Jumanji (1995),Adventure|Children|Fantasy,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1.732051
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1.414214
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,1.732051
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1.000000


In [7]:
df_movies_profile_normalization = df_movies_profile.iloc[:,3:]
for genre_column in df_movies_profile_normalization:
    df_movies_profile_normalization[genre_column] = df_movies_profile_normalization[genre_column] / df_movies_profile_normalization["norm"]
df_movies_profile_normalization.drop(columns=["norm"], inplace=True)

In [8]:
df_movies_profile_normalization.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,0.447214,0.447214,0.447214,0.447214,0.447214,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.577350,0.000000,0.577350,0.000000,0.577350,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.707107,0.000000,0.707107,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.577350,0.000000,0.577350,0.57735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
similarity_matrix = np.dot(df_movies_profile_normalization,df_movies_profile_normalization.T)
df_similarity_matrix = pd.DataFrame(similarity_matrix, index=df_movies['movieId'], columns=df_movies['movieId'])

In [10]:
df_similarity_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.774597,0.316228,0.258199,0.447214,0.000000,0.316228,0.632456,0.000000,0.258199,...,0.447214,0.316228,0.316228,0.447214,0.0,0.670820,0.774597,0.00000,0.316228,0.447214
2,0.774597,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.816497,0.000000,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.0,0.288675,0.333333,0.00000,0.000000,0.000000
3,0.316228,0.000000,1.000000,0.816497,0.707107,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.353553,0.000000,0.500000,0.000000,0.0,0.353553,0.408248,0.00000,0.000000,0.707107
4,0.258199,0.000000,0.816497,1.000000,0.577350,0.000000,0.816497,0.000000,0.000000,0.000000,...,0.288675,0.408248,0.816497,0.000000,0.0,0.288675,0.333333,0.57735,0.000000,0.577350
5,0.447214,0.000000,0.707107,0.577350,1.000000,0.000000,0.707107,0.000000,0.000000,0.000000,...,0.500000,0.000000,0.707107,0.000000,0.0,0.500000,0.577350,0.00000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.670820,0.288675,0.353553,0.288675,0.500000,0.288675,0.353553,0.000000,0.500000,0.288675,...,0.750000,0.353553,0.353553,0.500000,0.0,1.000000,0.866025,0.00000,0.707107,0.500000
193583,0.774597,0.333333,0.408248,0.333333,0.577350,0.000000,0.408248,0.000000,0.000000,0.000000,...,0.577350,0.408248,0.408248,0.577350,0.0,0.866025,1.000000,0.00000,0.408248,0.577350
193585,0.000000,0.000000,0.000000,0.577350,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.707107,0.707107,0.000000,0.0,0.000000,0.000000,1.00000,0.000000,0.000000


In [11]:
def get_top_k_similar_with_requirements(df_movies, df_ratings, df_similarity_matrix, min_rating=0, min_review_times=0, movieId=1, k=10, min_cos_score=0,max_cos_score=1):
    df_similar_movie = df_ratings.groupby('movieId').agg({"userId":"count","rating":"mean"}).reset_index()
    df_similar_movie.columns = ["movieId", "review times", "avg rating"]
    df_similar_movie = df_similar_movie.loc[(df_similar_movie["avg rating"]>=min_rating)&(df_similar_movie["review times"]>=min_review_times)]
    df_similar_movie = pd.merge(df_similar_movie,df_similarity_matrix[movieId],left_on="movieId",right_on="movieId")
    df_similar_movie.rename(columns = {movieId:"cos score"},inplace=True)
    df_similar_movie = df_similar_movie.loc[df_similar_movie["movieId"]!=movieId]
    df_similar_movie = df_similar_movie.loc[(df_similar_movie["cos score"]<=max_cos_score)&(df_similar_movie["cos score"]>=min_cos_score)]
    df_similar_movie = pd.merge(df_movies, df_similar_movie, on="movieId")
    df_similar_movie.sort_values(by=["cos score","avg rating","review times"],ascending=False,inplace=True)
    return df_similar_movie.head(k)

In [12]:
get_top_k_similar_with_requirements(df_movies, df_ratings, df_similarity_matrix,movieId=5938, min_review_times=2,max_cos_score=1)

,movieId,title,genres,review times,avg rating,cos score
4433,38061,Kiss Kiss Bang Bang (2005),Comedy|Crime|Mystery|Thriller,35,4.071429,1.000000
1601,2413,Clue (1985),Comedy|Crime|Mystery|Thriller,18,3.388889,1.000000
3039,5021,Murder by Death (1976),Comedy|Crime|Mystery|Thriller,3,2.833333,1.000000
2982,4898,Novocaine (2001),Comedy|Crime|Mystery|Thriller,4,2.625000,1.000000
610,911,Charade (1963),Comedy|Crime|Mystery|Romance|Thriller,13,3.807692,0.894427
4234,27674,11:14 (2003),Comedy|Crime|Drama|Mystery|Thriller,4,3.750000,0.894427
1170,1799,Suicide Kings (1997),Comedy|Crime|Drama|Mystery|Thriller,13,3.423077,0.894427
45,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,204,4.237745,0.866025
728,1089,Reservoir Dogs (1992),Crime|Mystery|Thriller,131,4.202290,0.866025
3080,5120,Sleuth (1972),Comedy|Mystery|Thriller,3,4.166667,0.866025
